In [11]:
import os
import sys
import csv
import numpy as np

if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")

import traci

sumoBinary = "C:/Program Files (x86)/Eclipse/Sumo/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "last_final.sumocfg"]

try:
    traci.start(sumoCmd)

    with open('waiting_times_summary2.csv', 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([
            'Step', 'Average Waiting Time', 'Sum Waiting Time', 
            'System Total Stopped', 'System Total Waiting Time', 
            'System Mean Waiting Time', 'System Mean Speed'
        ])

        step = 0
        while step < 300:
            traci.simulationStep()
            vehicle_ids = traci.vehicle.getIDList()
            speeds = [traci.vehicle.getSpeed(vid) for vid in vehicle_ids]
            waiting_times = [traci.vehicle.getWaitingTime(vid) for vid in vehicle_ids]

            sum_waiting_time = sum([traci.vehicle.getAccumulatedWaitingTime(vid) for vid in vehicle_ids])
            system_total_stopped = sum(int(speed < 0.1) for speed in speeds)
            system_total_waiting_time = sum(waiting_times)
            system_mean_waiting_time = np.mean(waiting_times) if vehicle_ids else 0
            system_mean_speed = np.mean(speeds) if vehicle_ids else 0

            if len(vehicle_ids) > 0:
                average_waiting_time = sum_waiting_time / len(vehicle_ids)
            else:
                average_waiting_time = 0

            if step % 5 == 0:
                print(f"Writing to CSV at step: {step}")  # Debugging output
                csv_writer.writerow([
                    step, average_waiting_time, sum_waiting_time, 
                    system_total_stopped, system_total_waiting_time, 
                    system_mean_waiting_time, system_mean_speed
                ])
                #avg sama sum waiting time bukan dari system
                csv_file.flush()  # Manually flush data to file

            step += 1

    traci.close()

except Exception as e:
    print("An error occurred:", e)
    # Check if the TraCI connection is still open and then close it
    if 'traci' in globals() and traci.connection.isConnected():
        traci.close()

Writing to CSV at step: 0
Writing to CSV at step: 5
Writing to CSV at step: 10
Writing to CSV at step: 15
Writing to CSV at step: 20
Writing to CSV at step: 25
Writing to CSV at step: 30
Writing to CSV at step: 35
Writing to CSV at step: 40
Writing to CSV at step: 45
Writing to CSV at step: 50
Writing to CSV at step: 55
Writing to CSV at step: 60
Writing to CSV at step: 65
Writing to CSV at step: 70
Writing to CSV at step: 75
Writing to CSV at step: 80
Writing to CSV at step: 85
Writing to CSV at step: 90
Writing to CSV at step: 95
Writing to CSV at step: 100
Writing to CSV at step: 105
Writing to CSV at step: 110
Writing to CSV at step: 115
Writing to CSV at step: 120
Writing to CSV at step: 125
Writing to CSV at step: 130
Writing to CSV at step: 135
Writing to CSV at step: 140
Writing to CSV at step: 145
Writing to CSV at step: 150
Writing to CSV at step: 155
Writing to CSV at step: 160
Writing to CSV at step: 165
Writing to CSV at step: 170
Writing to CSV at step: 175
Writing to CSV

In [ ]:
    traci.close()


In [ ]:
import os
import sys
import csv
import numpy as np
import traci

# ... [previous code for setup and TrafficSignal class]

try:
    traci.start(sumoCmd)
    ts_ids = list(traci.trafficlight.getIDList())

    # Initialize traffic_signals
    traffic_signals = {ts_id: TrafficSignal(ts_id) for ts_id in ts_ids}
    
    with open('waiting_times_summary2.csv', 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        
        # Prepare the header with additional columns for agent_info
        header = [
            'Step', 'Average Waiting Time', 'Sum Waiting Time',
            'System Total Stopped', 'System Total Waiting Time',
            'System Mean Waiting Time', 'System Mean Speed',
            'Total Stopped', 'Total Accumulated Waiting Time', 'Average Speed'
        ]
        csv_writer.writerow(header)
    
        step = 0
        while step < 300:
            traci.simulationStep()
            vehicle_ids = traci.vehicle.getIDList()
            speeds = [traci.vehicle.getSpeed(vid) for vid in vehicle_ids]
            waiting_times = [traci.vehicle.getWaitingTime(vid) for vid in vehicle_ids]

            sum_waiting_time = sum([traci.vehicle.getAccumulatedWaitingTime(vid) for vid in vehicle_ids])
            system_total_stopped = sum(int(speed < 0.1) for speed in speeds)
            system_total_waiting_time = sum(waiting_times)
            system_mean_waiting_time = np.mean(waiting_times) if vehicle_ids else 0
            system_mean_speed = np.mean(speeds) if vehicle_ids else 0

            if len(vehicle_ids) > 0:
                average_waiting_time = sum_waiting_time / len(vehicle_ids)
            else:
                average_waiting_time = 0

            if step % 5 == 0:
                agent_info = _get_per_agent_info(ts_ids, traffic_signals)
                
                # Combine system data with agent data
                row = [
                    step, average_waiting_time, sum_waiting_time,
                    system_total_stopped, system_total_waiting_time,
                    system_mean_waiting_time, system_mean_speed,
                    agent_info['agents_total_stopped'],
                    agent_info['agents_total_accumulated_waiting_time'],
                    np.mean([agent_info[f'{ts}_average_speed'] for ts in ts_ids])
                ]
    
                print(f"Writing to CSV at step: {step}")
                csv_writer.writerow(row)
                csv_file.flush()
    
            step += 1

    traci.close()

except Exception as e:
    print("An error occurred:", e)
    traci.close()
